**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-09 22:12:39,580 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 22:12:39.582 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-02-09 22:12:39.680 | INFO     | __main__:<module>:13 - User Role: publisher
2025-02-09 22:12:39.682 | INFO     | __main__:<module>:24 - Creating account with name: eager_ride
2025-02-09 22:12:39.683 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 22:12:39.704 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 22:12:39.705 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:12:39.706 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 22:12:39.707 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-02-09 22:12:39,730 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:12:39,732 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
202

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-09 22:12:39.815 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 22:12:39,818 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:12:39,835 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:12:39,837 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:12:42,817 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:12:42,826 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:12:42,828 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'd7a27ca5d11cfe4fd85051e8d3e2213d0ffe23b0baf658a1ba7e93313fecb084', 'timestamp': 1739139162828}
2025-02-09 22:12:42.829 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 22:12:42,833 - INFO - Appended data under 'eager_ride@test' in logs/account_data.json
2025-02-09 22:12:42.834 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 22:12:4

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 22:12:42,879 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 22:12:42.881 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-02-09 22:12:42.936 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 22:12:42.957 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 22:12:42.958 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 22:12:42.959 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 22:12:42.961 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-02-09 22:12:42,963 - INFO - Processing account: eager_ride@test
2025-02-09 22:12:42,964 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Eager Ride', 'foaf:mbox': 'eager_ride@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'College of Busniess Admnistration'}, 'schema:ident

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details)

2025-02-09 22:12:45.901 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 22:12:45,903 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 22:12:45,919 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 22:12:45,921 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 22:12:48,880 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 22:12:48,887 - INFO - ('COMMITTED', 5, 0)
2025-02-09 22:12:48,889 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'b46b526003c1716dbdb720871588db843457b30f0dc72225c3d7e1f110260784', 'timestamp': 1739139168889}
2025-02-09 22:12:48.891 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 22:12:48,895 - INFO - Appended data under 'eager_ride@test' in logs/account_data.json
2025-02-09 22:12:48.896 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 22:12:4

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-02-09 22:12:48.957 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'eager_ride@test', 'schema:publicKey': 'e69e7844c0fb9e698f40102a029addb68b3c45f7d2408a02240eda07f0049c97', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'eager_ride@email.com', 'foaf:name': 'Eager Ride', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'College of Busniess Admnistration'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '0887-1009-0790-X'}}
